In [157]:
#Importing libraries

library(tidyverse)
library(dplyr)
library(lubridate)
library(tidyr)
library(Hmisc)
library(gapminder)


In [158]:
#loading data set
eu_storedata <- readxl::read_xls("C:/Users/DELL/Downloads/Sample - EU Superstore.xls")


In [159]:
#saving date that will be used as current date("Checkdate")
Checkdate <-  as.POSIXct("2019-01-31")


In [160]:
#creating new orderID which is unique for each customer
df_eu_storedata <- eu_storedata |>  mutate(Order_ID = paste(`Order ID`, `Customer ID`, sep = '_'))


In [161]:
# View first six rows of data
head(eu_storedata)

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
<dbl>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,ES-2018-1311038,2018-02-07,2018-02-11,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,OFF-ST-10000988,Office Supplies,Storage,"Fellowes Folders, Blue",79.20,3,0,39.60
2,ES-2018-1311038,2018-02-07,2018-02-11,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,TEC-AC-10004144,Technology,Accessories,"SanDisk Numeric Keypad, Bluetooth",388.92,7,0,0.00
3,ES-2018-1311038,2018-02-07,2018-02-11,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,OFF-LA-10001915,Office Supplies,Labels,"Avery Legal Exhibit Labels, 5000 Label Set",35.19,3,0,16.11
4,ES-2018-1311038,2018-02-07,2018-02-11,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,OFF-ST-10004550,Office Supplies,Storage,"Fellowes Folders, Wire Frame",50.94,2,0,13.20
5,ES-2018-1311038,2018-02-07,2018-02-11,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,TEC-AC-10004068,Technology,Accessories,"Memorex Memory Card, USB",307.44,3,0,73.71
6,ES-2018-1311038,2018-02-07,2018-02-11,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,OFF-ST-10002271,Office Supplies,Storage,"Rogers Shelving, Wire Frame",122.40,2,0,37.92


In [162]:
# Columns and Rows of dataset
ncol(df_eu_storedata)
nrow(df_eu_storedata)

[1] 21

[1] 10000

In [163]:
# Columns and datatypes of sales data
sapply(df_eu_storedata, class)

$`Row ID`
[1] "numeric"

$`Order ID`
[1] "character"

$`Order Date`
[1] "POSIXct" "POSIXt" 

$`Ship Date`
[1] "POSIXct" "POSIXt" 

$`Ship Mode`
[1] "character"

$`Customer ID`
[1] "character"

$`Customer Name`
[1] "character"

$Segment
[1] "character"

$City
[1] "character"

$State
[1] "character"

$Country
[1] "character"

$Region
[1] "character"

$`Product ID`
[1] "character"

$Category
[1] "character"

$`Sub-Category`
[1] "character"

$`Product Name`
[1] "character"

$Sales
[1] "numeric"

$Quantity
[1] "numeric"

$Discount
[1] "numeric"

$Profit
[1] "numeric"

$Order_ID
[1] "character"

# Beginning Working for customer level metrics


In [164]:
Basic_metrics <-
  df_eu_storedata |>
  group_by(`Customer Name`) |> 
  summarise(Customer_ID = first(`Customer ID`),
            segment = first(Segment),
            total_profit = sum(Profit),
            average_order_value = (sum(Sales)/n_distinct(`Order ID`)),
            average_order_profit = (sum(Profit)/n_distinct(`Order ID`)),
            average_products_purchased_per_order = (n_distinct(`Product Name`)/n_distinct(`Order ID`)),
            Average_revenue_per_product_for_the_customer = (sum(Sales)/sum(Quantity)),
            Average_profit_per_product_for_the_customer = (sum(Profit))/sum(Quantity),
            total_revenue = sum(Sales), 
            total_transactions = n_distinct(`Order ID`),
            date_last_transaction = as.Date(max(`Order Date`)),
            days_last_transaction = round(as.numeric(difftime(Checkdate,max(`Order Date`),units = "days"))),
            date_first_transaction = as.Date(min(`Order Date`)))  

print(Basic_metrics)



# A tibble: 795 × 14
   Customer Na…¹ Custo…² segment total…³ avera…⁴ avera…⁵ avera…⁶ Avera…⁷ Avera…⁸
   <chr>         <chr>   <chr>     <dbl>   <dbl>   <dbl>   <dbl>   <dbl>   <dbl>
 1 Aaron Bergman AB-100… Consum…   1123.    510.    93.6    1.92    75.5   13.9 
 2 Aaron Hawkins AH-100… Corpor…    982.    497.   123.     1.25    94.6   23.4 
 3 Aaron Smayli… AS-100… Corpor…    422.    516.    32.5    1.85    70.6    4.44
 4 Adam Bellava… AB-100… Home O…   1020.    669.   204.     1.4    108.    32.9 
 5 Adam Hart     AH-100… Corpor…    528.    503.    58.7    2.11    62.8    7.34
 6 Adam Shillin… AS-100… Consum…    330.    517.    41.3    1.88    67.9    5.41
 7 Adrian Barton AB-101… Consum…    277.    492.    46.2    2.17    72.0    6.75
 8 Adrian Hane   AH-101… Home O…    906.    598.   151.     2.17    65.3   16.5 
 9 Adrian Shami  AS-101… Home O…    341.    824.   114.     1.67   155.    21.3 
10 Aimee Bixby   AB-101… Consum…   1010.   1523.   168.     2.17   150.    16.5 
# … wit

# working for RFM

In [165]:
RFM <- df_eu_storedata %>%
  group_by(`Customer Name`) %>%
  summarise(
    total_revenue = sum(Sales),
    total_transactions = n_distinct(`Order ID`),
    days_last_transaction = round(as.numeric(difftime(Checkdate, max(`Order Date`), units = "days")))
  ) %>%
  ungroup() %>%
  mutate(
    Recency_segment = cut2(days_last_transaction, g = 5),
    Frequency_segment = cut2(total_transactions, g = 5),
    Revenue_segment = cut2(total_revenue, g = 5)
  ) %>%
  mutate(
    Recency_score = as.integer(fct_rev(Recency_segment)),
    Frequency_score = as.integer(Frequency_segment),
    Revenue_score = as.integer(Revenue_segment)
  ) %>%
  mutate(
    RFMScore = Recency_score * 100 + Frequency_score * 10 + Revenue_score
  ) %>%
  mutate(
    Segment = case_when(
      RFMScore %in% c(555, 554, 544, 545, 454, 455, 445) ~ "Champions",
      RFMScore %in% c(543, 444, 435, 355, 354, 345, 344, 335) ~ "Loyal Customers",
      RFMScore %in% c(
        553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433,
        432, 423, 353, 352, 351, 342, 341, 333, 323
      ) ~ "Potential Loyalists",
      RFMScore %in% c(512, 511, 422, 421, 412, 411, 311) ~ "Recent Customers",
      RFMScore %in% c(
        525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313
      ) ~ "Promising",
      RFMScore %in% c(535, 534, 443, 434, 343, 334, 325, 324) ~ "Needs Attention",
      RFMScore %in% c(331, 321, 312, 221, 213) ~ "About to Sleep",
      RFMScore %in% c(
        255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145,
        143, 142, 135, 134, 133, 125, 124
      ) ~ "At Risk",
      RFMScore %in% c(155, 154, 144, 214, 215, 115, 114, 113) ~ "Can’t lose",
      RFMScore %in% c(
        332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211
      ) ~ "Hibernating",
      RFMScore %in% c(111, 112, 121, 131, 141, 151) ~ "Lost"
    )
  ) %>%
  select(`Customer Name`, Recency_score, Frequency_score, Revenue_score, RFMScore, Segment)

print(RFM)


# A tibble: 795 × 6
   `Customer Name`    Recency_score Frequency_score Revenue_sc…¹ RFMSc…² Segment
   <chr>                      <int>           <int>        <int>   <dbl> <chr>  
 1 Aaron Bergman                  1               5            5     155 Can’t …
 2 Aaron Hawkins                  4               4            4     444 Loyal …
 3 Aaron Smayling                 5               5            5     555 Champi…
 4 Adam Bellavance                2               2            3     223 Hibern…
 5 Adam Hart                      4               5            4     454 Champi…
 6 Adam Shillingsburg             5               4            4     544 Champi…
 7 Adrian Barton                  4               2            3     423 Potent…
 8 Adrian Hane                    3               2            3     323 Potent…
 9 Adrian Shami                   1               1            2     112 Lost   
10 Aimee Bixby                    3               2            5     325 Needs …
# … with

# Finding median days between transactions

In [166]:
Median_days_bw_transactions <- df_eu_storedata |>
  arrange(desc(`Order Date`)) |>
  group_by(`Customer Name`, `Order ID`) |>
  summarise(date_unique_combination = as.Date(first(`Order Date`))) |>
  arrange(desc(date_unique_combination)) |>
  mutate(diff_bw_days = as.integer(lag(date_unique_combination) - date_unique_combination)) |>
  arrange(desc(diff_bw_days)) |>
  group_by(`Customer Name`) |>
  summarise(median_days_bw_transaction = round(median(diff_bw_days, na.rm = TRUE)))
  
  print(Median_days_bw_transactions)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
   `Customer Name`    median_days_bw_transaction
   <chr>                                   <dbl>
 1 Aaron Bergman                              66
 2 Aaron Hawkins                              42
 3 Aaron Smayling                             86
 4 Adam Bellavance                           122
 5 Adam Hart                                 166
 6 Adam Shillingsburg                        157
 7 Adrian Barton                             181
 8 Adrian Hane                                81
 9 Adrian Shami                              137
10 Aimee Bixby                               270
# … with 785 more rows


# finding profitability group

In [167]:
profitability_group <- df_eu_storedata |> 
  group_by(`Customer ID`,`Customer Name`,`Product ID`) |> 
  summarise(profitt = sum(Profit)/sum(Quantity)) |> 
  group_by(`Customer ID`,`Customer Name`) |> 
  summarise(profitability = mean(profitt, na.rm = TRUE)) |> 
  ungroup() |>
  mutate(profitability_group = cut2(profitability, g=5) |> 
           factor(labels = c("Very Low", "Low", "Moderate", "High", "Very High"))) |> 
  select(`Customer ID`,`Customer Name`,profitability_group) 

print(profitability_group)

`summarise()` has grouped output by 'Customer ID', 'Customer Name'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'Customer ID'. You can override using the `.groups` argument.


# A tibble: 795 × 3
   `Customer ID` `Customer Name`      profitability_group
   <chr>         <chr>                <fct>              
 1 AA-10315      Alex Avila           Moderate           
 2 AA-10375      Allen Armold         Very Low           
 3 AA-10480      Andrew Allen         High               
 4 AA-10645      Anna Andreadi        Low                
 5 AB-10015      Aaron Bergman        Moderate           
 6 AB-10060      Adam Bellavance      Very High          
 7 AB-10105      Adrian Barton        Low                
 8 AB-10150      Aimee Bixby          High               
 9 AB-10165      Alan Barnes          High               
10 AB-10255      Alejandro Ballentine Moderate           
# … with 785 more rows


# finding shopper type

In [168]:
shopper_type <- df_eu_storedata |>
  group_by(`Customer Name`) |>
  summarise(undisc_quantity = sum(ifelse(Discount == 0,Quantity,0)), 
            discount_quantity =sum(ifelse(Discount>0,Quantity,0)), 
            undisc_purchase = sum(ifelse(Discount == 0,Sales,0)), 
            discount_purchase =sum(ifelse(Discount>0,Sales,0))) |> 
  mutate(shopper_type = case_when(
    discount_quantity > undisc_quantity & discount_purchase > undisc_purchase ~ "promotional_shopper",
    discount_purchase >= 2*undisc_purchase ~ "promotional_shopper",
    TRUE ~ "Regular_shopper")) |> 
  select(`Customer Name`,shopper_type)
print(shopper_type)

# A tibble: 795 × 2
   `Customer Name`    shopper_type       
   <chr>              <chr>              
 1 Aaron Bergman      Regular_shopper    
 2 Aaron Hawkins      Regular_shopper    
 3 Aaron Smayling     Regular_shopper    
 4 Adam Bellavance    Regular_shopper    
 5 Adam Hart          Regular_shopper    
 6 Adam Shillingsburg Regular_shopper    
 7 Adrian Barton      promotional_shopper
 8 Adrian Hane        promotional_shopper
 9 Adrian Shami       promotional_shopper
10 Aimee Bixby        promotional_shopper
# … with 785 more rows


# Finding most spent-on category and sub-category of each customer and finding sales percentage of that category

In [169]:
Mostspent_Category_SubCategory <- df_eu_storedata |> 
  group_by(`Customer Name`,Category,`Sub-Category`) |>
  summarise(max_sales = sum(Sales)) |> 
  group_by(`Customer Name`) |> 
  slice_max(max_sales, n = 1, with_ties = FALSE) |> 
  rename(sales_category = Category, sales_sub_category = `Sub-Category`) |> 
  left_join(
    df_eu_storedata  |> 
      group_by(`Customer Name`, Category) |> 
      mutate(category_sales = sum(Sales)) |> 
      group_by(`Customer Name`)  |> 
      summarise(Customer_sale = sum(Sales),
                percent_spending_on_sales_category = round((max(category_sales)/Customer_sale)*100)) |> 
      mutate(percent_spending_on_sales_category = paste0(percent_spending_on_sales_category, "%"))) |> 
  mutate(sales_category = paste0(percent_spending_on_sales_category, " ", sales_category)) |> 
  select(`Customer Name`,`sales_sub_category`,`sales_category`)

print(Mostspent_Category_SubCategory)

`summarise()` has grouped output by 'Customer Name', 'Category'. You can override using the `.groups` argument.
Joining with `by = join_by(`Customer Name`)`


# A tibble: 795 × 3
# Groups:   Customer Name [795]
   `Customer Name`    sales_sub_category sales_category     
   <chr>              <chr>              <chr>              
 1 Aaron Bergman      Copiers            80% Technology     
 2 Aaron Hawkins      Copiers            60% Technology     
 3 Aaron Smayling     Accessories        56% Technology     
 4 Adam Bellavance    Bookcases          47% Furniture      
 5 Adam Hart          Chairs             44% Furniture      
 6 Adam Shillingsburg Appliances         57% Office Supplies
 7 Adrian Barton      Storage            76% Office Supplies
 8 Adrian Hane        Storage            50% Office Supplies
 9 Adrian Shami       Bookcases          84% Furniture      
10 Aimee Bixby        Machines           85% Technology     
# … with 785 more rows


# Finding favorite category and Sub-Category of each customer on the basis of orders

In [170]:
Fav_category_SubCategory <-
df_eu_storedata |> 
  group_by(`Customer Name`,Category,`Sub-Category`) |>
  summarise(most_frequently_purchased = n_distinct(`Order ID`),
            most_sales_quantity = sum(Quantity),
            most_sales = sum(Sales)) |> 
  group_by(`Customer Name`) |> 
  slice_max(most_frequently_purchased, n = 1, with_ties = TRUE) |> 
  slice_max(most_sales_quantity, n = 1, with_ties = TRUE) |> 
  slice_max(most_sales, n = 1, with_ties = TRUE) |> 
  select(`Customer Name`, Category,`Sub-Category`) |> 
  rename(most_frequently_purchased_category = Category, most_frequently_purchased_sub_category = `Sub-Category`)
print(Fav_category_SubCategory)

`summarise()` has grouped output by 'Customer Name', 'Category'. You can override using the `.groups` argument.


# A tibble: 795 × 3
# Groups:   Customer Name [795]
   `Customer Name`    most_frequently_purchased_category most_frequently_purch…¹
   <chr>              <chr>                              <chr>                  
 1 Aaron Bergman      Office Supplies                    Labels                 
 2 Aaron Hawkins      Office Supplies                    Art                    
 3 Aaron Smayling     Office Supplies                    Art                    
 4 Adam Bellavance    Office Supplies                    Binders                
 5 Adam Hart          Office Supplies                    Binders                
 6 Adam Shillingsburg Office Supplies                    Art                    
 7 Adrian Barton      Office Supplies                    Storage                
 8 Adrian Hane        Office Supplies                    Storage                
 9 Adrian Shami       Furniture                          Bookcases              
10 Aimee Bixby        Technology                         

# finding out favorite ship mode of each customer on basis of two factors. 1st(frequently order placed) 2nd(most quantity bought)


In [171]:
ship_mode <- df_eu_storedata |> 
  group_by(`Customer Name`,`Ship Mode`) |> 
  summarise(n_Order_Id = n_distinct(`Order ID`),
            max_qty = sum(Quantity)) |> 
  group_by(`Customer Name`) |> 
  slice_max(n_Order_Id, n = 1, with_ties = TRUE) |> 
  slice_max(max_qty, n = 1, with_ties = FALSE) |>
  select(`Customer Name`,`Ship Mode`)
print(ship_mode)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
# Groups:   Customer Name [795]
   `Customer Name`    `Ship Mode`   
   <chr>              <chr>         
 1 Aaron Bergman      Standard Class
 2 Aaron Hawkins      Standard Class
 3 Aaron Smayling     Standard Class
 4 Adam Bellavance    Second Class  
 5 Adam Hart          Standard Class
 6 Adam Shillingsburg Standard Class
 7 Adrian Barton      Standard Class
 8 Adrian Hane        Standard Class
 9 Adrian Shami       Second Class  
10 Aimee Bixby        Standard Class
# … with 785 more rows


# Analyzing base region of a customer with conditions, 1st(frequent order), 2nd(Most Quantity)


In [172]:
Region <-
df_eu_storedata |> 
  group_by(`Customer Name`,`Region`) |> 
  summarise(n_order_Id = n_distinct(`Order ID`),
            max_qty = sum(Quantity)) |> 
  group_by(`Customer Name`) |> 
  slice_max(n_order_Id, n = 1, with_ties = TRUE) |> 
  slice_max(max_qty, n = 1, with_ties = FALSE) |> 
  select(`Customer Name`,`Region`)
print(Region)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
# Groups:   Customer Name [795]
   `Customer Name`    Region 
   <chr>              <chr>  
 1 Aaron Bergman      Central
 2 Aaron Hawkins      Central
 3 Aaron Smayling     Central
 4 Adam Bellavance    North  
 5 Adam Hart          Central
 6 Adam Shillingsburg Central
 7 Adrian Barton      South  
 8 Adrian Hane        North  
 9 Adrian Shami       Central
10 Aimee Bixby        North  
# … with 785 more rows


# Analyzing base country of a customer with conditions, 1st(frequent order), 2nd(Most Qty), 3rd(minimum variability in state), 4th(minimum variability in city), 5th(maximum sales)
 

In [173]:
Country <- 
  df_eu_storedata |> 
  group_by(`Customer Name`,Country) |> 
  summarise(n_order_Id = n_distinct(`Order ID`),
            max_qty = sum(Quantity),
            n_state = n_distinct(State),
            n_city = n_distinct(City),
            sale = sum(Sales)) |> 
  group_by(`Customer Name`) |> 
  slice_max(n_order_Id, n = 1, with_ties = TRUE) |> 
  slice_max(max_qty, n = 1, with_ties = TRUE) |> 
  slice_min(n_state, n = 1, with_ties = TRUE) |> 
  slice_min(n_city, n = 1, with_ties = TRUE) |> 
  slice_max(sale, n = 1, with_ties = TRUE) |>
  select(`Customer Name`,`Country`)
print(Country)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
# Groups:   Customer Name [795]
   `Customer Name`    Country       
   <chr>              <chr>         
 1 Aaron Bergman      Italy         
 2 Aaron Hawkins      United Kingdom
 3 Aaron Smayling     France        
 4 Adam Bellavance    United Kingdom
 5 Adam Hart          France        
 6 Adam Shillingsburg France        
 7 Adrian Barton      Italy         
 8 Adrian Hane        United Kingdom
 9 Adrian Shami       Netherlands   
10 Aimee Bixby        United Kingdom
# … with 785 more rows


# Analyzing base state of a customer with conditions, 1st(frequent order), 2nd(Most Qty), 3rd(minimum variability in city), 4th(maximum sales)
 

In [174]:
State <- 
  df_eu_storedata |> 
  group_by(`Customer Name`,State) |> 
  summarise(n_order_Id = n_distinct(`Order ID`),
            max_qty = sum(Quantity),
            n_city = n_distinct(City),
            sale = sum(Sales)) |> 
  group_by(`Customer Name`) |> 
  slice_max(n_order_Id, n = 1, with_ties = TRUE) |> 
  slice_max(max_qty, n = 1, with_ties = TRUE) |> 
  slice_min(n_city, n = 1, with_ties = TRUE) |> 
  slice_max(sale, n = 1, with_ties = TRUE) |>
  select(`Customer Name`,`State`)
print(State)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
# Groups:   Customer Name [795]
   `Customer Name`    State          
   <chr>              <chr>          
 1 Aaron Bergman      England        
 2 Aaron Hawkins      England        
 3 Aaron Smayling     England        
 4 Adam Bellavance    Oslo           
 5 Adam Hart          England        
 6 Adam Shillingsburg Ile-de-France  
 7 Adrian Barton      Hauts-de-France
 8 Adrian Hane        England        
 9 Adrian Shami       Overijssel     
10 Aimee Bixby        England        
# … with 785 more rows


# Analyzing base city of a customer with conditions, 1st(frequent order), 2nd(Most Qty), 3rd(maximum sales)

In [175]:
City <- 
  df_eu_storedata |> 
  group_by(`Customer Name`,City) |> 
  summarise(n_order_Id = n_distinct(`Order ID`),
            max_qty = sum(Quantity),
            sale = sum(Sales)) |> 
  group_by(`Customer Name`) |> 
  slice_max(n_order_Id, n = 1, with_ties = TRUE) |> 
  slice_max(max_qty, n = 1, with_ties = TRUE) |> 
  slice_max(sale, n = 1, with_ties = TRUE) |>
  select(`Customer Name`,`City`)
print(City)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
# Groups:   Customer Name [795]
   `Customer Name`    City          
   <chr>              <chr>         
 1 Aaron Bergman      Viersen       
 2 Aaron Hawkins      Maisons-Alfort
 3 Aaron Smayling     Le Cannet     
 4 Adam Bellavance    Oslo          
 5 Adam Hart          Roeselare     
 6 Adam Shillingsburg Huddersfield  
 7 Adrian Barton      Avion         
 8 Adrian Hane        London        
 9 Adrian Shami       Deventer      
10 Aimee Bixby        London        
# … with 785 more rows


# Finding Average shipping time for the customer


In [176]:
Avg_time <- df_eu_storedata |> 
  group_by(`Customer Name`,`Order ID`) |> 
  summarise(S = difftime(`Ship Date`,`Order Date`)) |> 
  group_by(`Customer Name`) |> 
  summarise(avg_shipping_hours = round(mean(as.numeric(S)/3600)))
print(Avg_time)

Warning message:
"Returning more (or less) than 1 row per `summarise()` group was deprecated in dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()` always returns an ungrouped data frame
  and adjust accordingly."
`summarise()` has grouped output by 'Customer Name', 'Order ID'. You can override using the `.groups` argument.


# A tibble: 795 × 2
   `Customer Name`    avg_shipping_hours
   <chr>                           <dbl>
 1 Aaron Bergman                      74
 2 Aaron Hawkins                     101
 3 Aaron Smayling                    108
 4 Adam Bellavance                    89
 5 Adam Hart                         108
 6 Adam Shillingsburg                 94
 7 Adrian Barton                      94
 8 Adrian Hane                        79
 9 Adrian Shami                       96
10 Aimee Bixby                        81
# … with 785 more rows


# Percentage of orders with multiple products


In [177]:
products_per_order <- df_eu_storedata |> 
  group_by(`Customer ID`,`Customer Name`,`Order ID`) |> 
  summarise(num_products = case_when(n_distinct(`Product ID`) > 1 ~ "Multiple", TRUE ~ "Single")) |> 
  group_by(`Customer Name`) |> 
  summarise(multiple_products_per_order_percentage = (sum(case_when(num_products == "Multiple" ~ 1, TRUE ~ 0))/n_distinct(`Order ID`))*100)
print(products_per_order)

`summarise()` has grouped output by 'Customer ID', 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 2
   `Customer Name`    multiple_products_per_order_percentage
   <chr>                                               <dbl>
 1 Aaron Bergman                                        75  
 2 Aaron Hawkins                                        25  
 3 Aaron Smayling                                       30.8
 4 Adam Bellavance                                      20  
 5 Adam Hart                                            66.7
 6 Adam Shillingsburg                                   50  
 7 Adrian Barton                                        83.3
 8 Adrian Hane                                          83.3
 9 Adrian Shami                                         66.7
10 Aimee Bixby                                          66.7
# … with 785 more rows


# Calculating the year-over-year(last year) growth in sales for each customer


In [178]:
sales_growth <- df_eu_storedata |> 
  group_by(`Customer ID`,`Customer Name`,year = lubridate::year(`Order Date`)) |> 
  summarise(total_sales = sum(`Sales`)) |> 
  group_by(`Customer ID`) |> 
  mutate(growth_rate = (total_sales / lag(total_sales) - 1) * 100) |> 
  filter(year == last(year)) |> 
  select(`Customer ID`,`Customer Name`,year,growth_rate)
print(sales_growth)

`summarise()` has grouped output by 'Customer ID', 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 795 × 4
# Groups:   Customer ID [795]
   `Customer ID` `Customer Name`       year growth_rate
   <chr>         <chr>                <dbl>       <dbl>
 1 AA-10315      Alex Avila            2018       610. 
 2 AA-10375      Allen Armold          2018       -70.8
 3 AA-10480      Andrew Allen          2018        22.1
 4 AA-10645      Anna Andreadi         2018       344. 
 5 AB-10015      Aaron Bergman         2018       274. 
 6 AB-10060      Adam Bellavance       2018       -20.2
 7 AB-10105      Adrian Barton         2018       826. 
 8 AB-10150      Aimee Bixby           2018       111. 
 9 AB-10165      Alan Barnes           2018       -35.3
10 AB-10255      Alejandro Ballentine  2018       -45.1
# … with 785 more rows


# Combining all metrics by left join

In [179]:
EUstore <- left_join(Median_days_bw_transactions, RFM, by = "Customer Name") %>%
  left_join(Fav_category_SubCategory, by = "Customer Name") |> 
  left_join(Mostspent_Category_SubCategory, by = "Customer Name") |> 
  left_join(profitability_group, by = "Customer Name") |> 
  left_join(shopper_type, by = "Customer Name") |> 
  left_join(sales_growth, by = "Customer Name") |> 
  left_join(products_per_order, by = "Customer Name") |> 
  left_join(Region, by = "Customer Name") |> 
  left_join(Country, by = "Customer Name") |> 
  left_join(State, by = "Customer Name") |> 
  left_join(City, by = "Customer Name") |> 
  left_join(ship_mode, by = "Customer Name") |> 
  left_join(Avg_time, by = "Customer Name") |> 
  left_join(Basic_metrics, by = "Customer Name")
print(EUstore)

# A tibble: 795 × 37
   Customer Na…¹ media…² Recen…³ Frequ…⁴ Reven…⁵ RFMSc…⁶ Segment most_…⁷ most_…⁸
   <chr>           <dbl>   <int>   <int>   <int>   <dbl> <chr>   <chr>   <chr>  
 1 Aaron Bergman      66       1       5       5     155 Can’t … Office… Labels 
 2 Aaron Hawkins      42       4       4       4     444 Loyal … Office… Art    
 3 Aaron Smayli…      86       5       5       5     555 Champi… Office… Art    
 4 Adam Bellava…     122       2       2       3     223 Hibern… Office… Binders
 5 Adam Hart         166       4       5       4     454 Champi… Office… Binders
 6 Adam Shillin…     157       5       4       4     544 Champi… Office… Art    
 7 Adrian Barton     181       4       2       3     423 Potent… Office… Storage
 8 Adrian Hane        81       3       2       3     323 Potent… Office… Storage
 9 Adrian Shami      137       1       1       2     112 Lost    Furnit… Bookca…
10 Aimee Bixby       270       3       2       5     325 Needs … Techno… Machin…
# … wit